In [50]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import ec

# Generate ECC private key
def generate_ecc_private_key():
    private_key = ec.generate_private_key(ec.SECP192R1(), default_backend())
    return private_key

# Generate ECDH shared key
def generate_ecdh_shared_key(private_key, peer_public_key_bytes):
    peer_public_key = serialization.load_pem_public_key(peer_public_key_bytes, backend=default_backend())
    shared_key = private_key.exchange(ec.ECDH(), peer_public_key)
    return shared_key

# Example usage
# Alice generates her private key
clientPrivateKey = generate_ecc_private_key()
# Bob generates his private key
# bob_private_key = generate_ecc_private_key()

# Alice sends her public key to Bob
clientPublicKeyBytes = clientPrivateKey.public_key().public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

# # Bob sends his public key to Alice
# bob_public_key_bytes = bob_private_key.public_key().public_bytes(
#     encoding=serialization.Encoding.PEM,
#     format=serialization.PublicFormat.SubjectPublicKeyInfo
# )

# Alice computes the shared key using Bob's public key
# alice_shared_key = generate_ecdh_shared_key(alice_private_key, bob_public_key_bytes)

# Bob computes the shared key using Alice's public key
# bob_shared_key = generate_ecdh_shared_key(bob_private_key, alice_public_key_bytes)

# The shared keys should match
# assert alice_shared_key == bob_shared_key
# print("Shared Key:", alice_shared_key.hex())
randomNo = ''
serverPubKey=''

In [51]:
# def checkResp(Res,rq_ResFn,rq_ResSFn):
#     Res = Res.decode()
#     Res_Fn = Res[0]+Res[1]
#     Res_SFn = Res[3]+Res[4]
# a= 8
# len(str(a))
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
def encrypt_AES(Cryptkey, data):
#     print(data)
#     print(len(data))
    data = bytes.fromhex(data)
    cipher = AES.new(Cryptkey, AES.MODE_CBC)
    ciphertext = cipher.encrypt(pad(data, AES.block_size))
    iv = cipher.iv
    return iv + ciphertext

def decrypt_AES(key, encrypted_data):
    iv = encrypted_data[:AES.block_size]
    ciphertext = encrypted_data[AES.block_size:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphertext), AES.block_size)
    return plaintext

# Example usage
# key = get_random_bytes(16)  # 16 bytes key for AES-128
Cryptkey = b'\xb9t5RR\x9d\x1d+\xaa\xa3\xd1}\xfb\x1a\x1d\xcb'
# data = b"Hello, world! This is AES encryption example."

# encrypted_data = encrypt_AES(key, data)
# print("Encrypted data:", encrypted_data)

# decrypted_data = decrypt_AES(key, encrypted_data)
# print("Decrypted data:", decrypted_data.decode())

In [52]:
def KeyGen(seed):
    global Cryptkey
    return encrypt_AES(Cryptkey, seed).hex()
def ValLevel(lvl):
    if len(lvl)==1:
        return '0'+lvl
    else:
        return lvl
def ExtractKS (from_server, secAccLvl):
#     print(from_server[0:2])
    global randomNo , serverPubKey, clientPrivateKey
    if from_server[0:2]  == '67' and int(from_server[3:5])%2 == 0: 
        print(from_server)
        print('ECU unlocked sucessfully')
        return 1
    if from_server[0:2]  == '67' and int(from_server[3:5])%2 == 1:
        serverPubKey = bytes.fromhex(from_server[15:len(from_server)])
        randomNo = generate_ecdh_shared_key(clientPrivateKey,serverPubKey)
        return '27 ' + ValLevel(str(secAccLvl+1)) + ' '+KeyGen(randomNo.hex()+from_server[6:14])
    return -1
# print(ExtractKS('67 01 67 68 69'))

In [53]:
import socket
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(('127.0.0.1', 8080))
closeCon = 0
client.send("Tester Connected\n".encode())
secAccLvl = '07'
while closeCon ==0 :
    global clientPublicKeyBytes
    from_server = client.recv(4096)
    if from_server.decode() == 'End':
        closeCon = 1
        break
    print (from_server.decode())
    if int(secAccLvl)%2 == 1:
        resp = '27 '+ secAccLvl + ' ' + clientPublicKeyBytes.hex()
        print(resp)
        client.send(resp.encode())
        from_server = client.recv(4096).decode()
        print (from_server)
    resp = ExtractKS(from_server,int(secAccLvl))
    if resp == -1:
        print('error in message Fn or SubFn')
        closeCon = 1
        client.send(resp.encode())
        break
    elif resp == 1:
        print("ECU Unlocked")
        closeCon = 1
        client.send(resp.encode())
        break
    else:
        secAccLvl = str(int(secAccLvl)+1)
        closeCon = 1
        print(resp)
        client.send(resp.encode())
        from_server = client.recv(4096).decode()
        resp = ExtractKS(from_server,int(secAccLvl))
client.close()



ECU Connected
27 07 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741456a414f7754446c414141425a453962454371794a6c773577356c6e360a7947786e5647382b6f4f4e6c6d4738753468766656435739394a35695663467276307a6d0a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a
67 07 b9af3ead 2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d456b77457759484b6f5a497a6a3043415159494b6f5a497a6a3044415145444d6741455861687171684d75336b7059483039735449324266754134345135370a62414948354b73656f6f4f384b394a5a5a3045414e4970554e2f597a4c6b69706c6e64310a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a
27 08 ace35dbdea5117ee281c43f892ab0b897c6db0ce9d745b4ecf94e51cf7777027cf6803077a82de7c3f5c78d8c037943b
67 08
ECU unlocked sucessfully


In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import ec
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import socket

def generate_ecc_private_key():
    private_key = ec.generate_private_key(ec.SECP192R1(), default_backend())
    return private_key

def generate_ecdh_shared_key(private_key, peer_public_key_bytes):
    peer_public_key = serialization.load_pem_public_key(peer_public_key_bytes, backend=default_backend())
    shared_key = private_key.exchange(ec.ECDH(), peer_public_key)
    return shared_key

clientPrivateKey = generate_ecc_private_key()

clientPublicKeyBytes = clientPrivateKey.public_key().public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

randomNo = ''
serverPubKey=''
def encrypt_AES(Cryptkey, data):
#     print(data)
#     print(len(data))
    data = bytes.fromhex(data)
    cipher = AES.new(Cryptkey, AES.MODE_CBC)
    ciphertext = cipher.encrypt(pad(data, AES.block_size))
    iv = cipher.iv
    return iv + ciphertext

def decrypt_AES(key, encrypted_data):
    iv = encrypted_data[:AES.block_size]
    ciphertext = encrypted_data[AES.block_size:]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = unpad(cipher.decrypt(ciphertext), AES.block_size)
    return plaintext

Cryptkey = b'\xb9t5RR\x9d\x1d+\xaa\xa3\xd1}\xfb\x1a\x1d\xcb'

def KeyGen(seed):
    global Cryptkey
    return encrypt_AES(Cryptkey, seed).hex()
def ValLevel(lvl):
    if len(lvl)==1:
        return '0'+lvl
    else:
        return lvl
def ExtractKS (from_server, secAccLvl):
    global randomNo , serverPubKey, clientPrivateKey
    if from_server[0:2]  == '67' and int(from_server[3:5])%2 == 0: 
        print(from_server)
        print('ECU unlocked sucessfully')
        return 1
    if from_server[0:2]  == '67' and int(from_server[3:5])%2 == 1:
        serverPubKey = bytes.fromhex(from_server[15:len(from_server)])
        randomNo = generate_ecdh_shared_key(clientPrivateKey,serverPubKey)
        return '27 ' + ValLevel(str(secAccLvl+1)) + ' '+KeyGen(randomNo.hex()+from_server[6:14])
    return -1
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(('127.0.0.1', 8080))
closeCon = 0
client.send("Tester Connected\n".encode())
secAccLvl = '07'
while closeCon ==0 :
    global clientPublicKeyBytes
    from_server = client.recv(4096)
    if from_server.decode() == 'End':
        closeCon = 1
        break
    print (from_server.decode())
    if int(secAccLvl)%2 == 1:
        resp = '27 '+ secAccLvl + ' ' + clientPublicKeyBytes.hex()
        print(resp)
        client.send(resp.encode())
        from_server = client.recv(4096).decode()
        print (from_server)
    resp = ExtractKS(from_server,int(secAccLvl))
    if resp == -1:
        print('error in message Fn or SubFn')
        closeCon = 1
        client.send(resp.encode())
        break
    elif resp == 1:
        print("ECU Unlocked")
        closeCon = 1
        client.send(resp.encode())
        break
    else:
        secAccLvl = str(int(secAccLvl)+1)
        closeCon = 1
        print(resp)
        client.send(resp.encode())
        from_server = client.recv(4096).decode()
        resp = ExtractKS(from_server,int(secAccLvl))
client.close()

